In [1]:

import findspark
import os
import configparser
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
import pymongo

## Spark Session is entry point to SPARK SQL

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [12]:
spark

In [4]:
empDF = spark.read.json("dataset/customerData.json")

In [8]:
empDF.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [9]:
empDF.printSchema()

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [11]:
#results 
empDF.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [13]:
empDF.filter(empDF["age"] == 40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



In [14]:
empDF.groupBy("deptid").agg({"salary": "avg", "age": "max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



In [15]:
## create a data frome from the list

In [16]:
deptList = [{'name': 'Sales', 'id':"100"},
            {'name': 'Engineering', 'id':"200"}]

In [17]:
deptList

[{'name': 'Sales', 'id': '100'}, {'name': 'Engineering', 'id': '200'}]

In [18]:
deptDF = spark.createDataFrame(deptList)

C:\Spark\spark-2.4.6-bin-hadoop2.6\python\pyspark\sql\session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [19]:
deptDF.show()

+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [20]:
empDF.join(deptDF, empDF.deptid == deptDF.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+



In [21]:
empDF.filter(empDF["age"] > 30)\
    .join(deptDF, empDF.deptid == deptDF.id)\
    .groupBy("deptid")\
    .agg({"salary":"avg", "age": "max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



In [29]:
from pyspark.sql import Row

In [33]:
autoDF1 = spark.read.csv("dataset/auto-data.csv", header=True)

In [34]:
autoDF1.show()

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  fwd|     four| 69|4900|      31|     36| 5118|
| chevrolet|     gas|   std|  two|hatchback|  fwd|    three| 48|5100|      47|     53| 5151|
|     mazda|     gas|   std|  two|hatchback|  fwd|     four| 68|5000|      30|     31| 5195|
|    toyota|     gas|   std|  two|hatchback|  fwd|     four| 62|4800|      35|     39| 5348|
|mitsubishi|     gas|   std|  two|hatchback|  fwd|     four| 68|5500|      37|     41| 5389|
|     honda|     gas|   std|  two|hatchback|  fwd|     four| 60|5500|      38|     42| 5399|
|    nissan|     gas|   std|  two|    sedan|  fwd|     four| 69|5200|      31|     37| 5499|
|     dodge|     gas|   std|  two|hatchback|  fwd|     four| 68|5500| 

## Temp table: Provide SQL table like operations
- createOrReplaceTempView: register the DataFrame as a table within SQL Session

In [37]:
autoDF1.createOrReplaceTempView('autos')

In [39]:
spark.sql("select * from autos where hp > 200").show()

+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+
|   MAKE|FUELTYPE|ASPIRE|DOORS|       BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+
|porsche|     gas|   std|  two|    hardtop|  rwd|      six|207|5900|      17|     25|32528|
|porsche|     gas|   std|  two|    hardtop|  rwd|      six|207|5900|      17|     25|34028|
| jaguar|     gas|   std|  two|      sedan|  rwd|   twelve|262|5000|      13|     17|36000|
|porsche|     gas|   std|  two|convertible|  rwd|      six|207|5900|      17|     25|37028|
+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+



## Convert Spark DF to python DF

In [41]:
empPandas = empDF.toPandas()

In [42]:
empPandas

,age,deptid,gender,name,salary
0,32,100,male,Benjamin Garrison,3000
1,40,200,male,Holland Drake,4500
2,26,100,male,Burks Velasquez,2700
3,51,100,female,June Rutledge,4300
4,44,200,male,Nielsen Knapp,6500


## Loading Database from JDBC source

In [ ]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

In [ ]:
jdbcDF2 = spark.read \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

## writing or saving to JDBC source

In [ ]:
jdbcDF.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .save()

In [ ]:
jdbcDF2.write \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

## References:
- https://spark.apache.org/docs/latest/sql-data-sources-hive-tables.html